In [4]:
from datasets import load_dataset

# Load only the Python subset from CodeSearchNet
dataset = load_dataset("code_search_net", "python")

In [7]:
dataset = dataset["train"]

In [14]:
include_metadata = ["func_documentation_string", "func_code_string", "func_name", "func_code_url"]
df = pd.DataFrame(dataset)[include_metadata]

In [27]:
#Lightly process the func_code_string value. Remove trailing whitespaces and extra lines
import re
def preprocess_func_code_string(code):
    code = "\n".join([line.rstrip() for line in code.splitlines()])
    code = re.sub(r'n\{2,}', '\n\n', code)
    return code

In [25]:
df[:5]["func_code_string"][0]

'def __msgc_step3_discontinuity_localization(self):\n        """\n        Estimate discontinuity in basis of low resolution image segmentation.\n        :return: discontinuity in low resolution\n        """\n        import scipy\n\n        start = self._start_time\n        seg = 1 - self.segmentation.astype(np.int8)\n        self.stats["low level object voxels"] = np.sum(seg)\n        self.stats["low level image voxels"] = np.prod(seg.shape)\n        # in seg is now stored low resolution segmentation\n        # back to normal parameters\n        # step 2: discontinuity localization\n        # self.segparams = sparams_hi\n        seg_border = scipy.ndimage.filters.laplace(seg, mode="constant")\n        logger.debug("seg_border: %s", scipy.stats.describe(seg_border, axis=None))\n        # logger.debug(str(np.max(seg_border)))\n        # logger.debug(str(np.min(seg_border)))\n        seg_border[seg_border != 0] = 1\n        logger.debug("seg_border: %s", scipy.stats.describe(seg_border, a

In [28]:
df["func_code_string"].apply(preprocess_func_code_string)

0         def __msgc_step3_discontinuity_localization(se...
1         def __multiscale_gc_lo2hi_run(self):  # , pyed...
2         def __multiscale_gc_hi2lo_run(self):  # , pyed...
3         def __ordered_values_by_indexes(self, data, in...
4         def __hi2lo_multiscale_indexes(self, mask, ori...
                                ...                        
412173    def load_object(obj) -> object:\n    """Load a...
412174    def escape_args(cls, *args):\n        """\n   ...
412175    def get_page_from_string_or_id(page_string, la...
412176    def pages_siblings_menu(context, page, url='/'...
412177    def pages_admin_menu(context, page):\n    """R...
Name: func_code_string, Length: 412178, dtype: object

In [29]:
df[:5]["func_code_string"][0]

'def __msgc_step3_discontinuity_localization(self):\n        """\n        Estimate discontinuity in basis of low resolution image segmentation.\n        :return: discontinuity in low resolution\n        """\n        import scipy\n\n        start = self._start_time\n        seg = 1 - self.segmentation.astype(np.int8)\n        self.stats["low level object voxels"] = np.sum(seg)\n        self.stats["low level image voxels"] = np.prod(seg.shape)\n        # in seg is now stored low resolution segmentation\n        # back to normal parameters\n        # step 2: discontinuity localization\n        # self.segparams = sparams_hi\n        seg_border = scipy.ndimage.filters.laplace(seg, mode="constant")\n        logger.debug("seg_border: %s", scipy.stats.describe(seg_border, axis=None))\n        # logger.debug(str(np.max(seg_border)))\n        # logger.debug(str(np.min(seg_border)))\n        seg_border[seg_border != 0] = 1\n        logger.debug("seg_border: %s", scipy.stats.describe(seg_border, a

In [30]:
#loading CodeBERT
!pip install transformers

from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
#Tokenizing and appending raw tokens to the dataframe. 
def tokenize_func_code_string(code):
    return tokenizer.encode(code, truncation=True, padding="max_length") #for consistent token length and code too long for tokenizer

df["func_code_string_token"] = df["func_code_string"].apply(tokenize_func_code_string)